# Sakila DB Project

`Sakila` is a sample database created by `MySql` [Link](https://dev.mysql.com/doc/sakila/en/)

by 
Deswandhi
deswandhi@gmail.com


# 1. Data Warehouse

## 1.1 Connect to local database

In [1]:
# Import necessary modules
import pandas as pd
import pymysql

# connect to the sakila database
db_engine = pymysql.connect(host='localhost', user='deswandhi', password='@El13210017', database='sakila', use_unicode=True)


## 1.2 Dimensional Model

### 1.2.1 Fact

In [2]:
### Fact data table
fact_rent = pd.read_sql("""SELECT customer_id, staff_id, inventory.film_id, category.name AS category, inventory.store_id, rental_date, return_date, SUM(rental_duration), rental_id
                        FROM rental
                        JOIN inventory on rental.inventory_id = inventory.inventory_id
                        JOIN film on inventory.film_id = film.film_id
                        JOIN film_category on  film.film_id = film_category.film_id
                        JOIN category on film_category.category_id = category.category_id
                        GROUP BY customer_id;""", db_engine)


print(fact_rent.head())



   customer_id  staff_id  film_id  store_id         rental_date  \
0          431         2        1         1 2005-07-08 19:03:15   
1          518         1        1         1 2005-08-02 20:13:10   
2          279         1        1         1 2005-08-21 21:27:43   
3          411         1        1         1 2005-05-30 20:21:07   
4          170         2        1         1 2005-06-17 20:24:00   

          return_date  rental_duration  rental_id  
0 2005-07-11 21:29:15                6       4863  
1 2005-08-11 21:35:10                6      11433  
2 2005-08-30 22:26:43                6      14714  
3 2005-06-06 00:36:07                6        972  
4 2005-06-23 17:45:00                6       2117  


### 1.2.2 Dimensions

In [4]:
### Dimension data table
# What => Film
dim_film = pd.read_sql("""SELECT film.film_id, title, rating, language.name AS language, category.name AS category
                            FROM film
                            JOIN language on film.language_id = language.language_id
                            JOIN film_category on  film.film_id = film_category.film_id
                            JOIN category on film_category.category_id = category.category_id;
                            """, db_engine)

# Where => Store
dim_store = pd.read_sql("""SELECT customer_id, staff_id, inventory.film_id, inventory.store_id, rental_date, return_date, SUM(rental_duration) AS rental_duration, rental_id
                        FROM rental
                        JOIN inventory on rental.inventory_id = inventory.inventory_id
                        JOIN film on inventory.film_id = film.film_id
                        GROUP BY customer_id;
                        """, db_engine)


# Who => Staff
dim_staff = pd.read_sql("""SELECT staff_id,  store.store_id, CONCAT(first_name,' ', last_name) AS staff_name, email
                         FROM staff
                        JOIN store on staff.staff_id = store.manager_staff_id;
                        """, db_engine)

# Who => Customer
dim_customer = pd.read_sql("""SELECT customer_id, CONCAT(first_name,' ', last_name) AS customer_name, active, address AS customer_address, city AS customer_city, country AS customer_country
                            FROM customer
                            JOIN address on customer.address_id = address.address_id
                            JOIN city on address.city_id = city.city_id
                            JOIN country on city.country_id = country.country_id;
                             """, db_engine)

print(dim_film.head(2))
print(dim_store.head(2))
print(dim_staff.head(2))
print(dim_customer.head(2))

   film_id            title rating language category
0       19     AMADEUS HOLY     PG  English   Action
1       21  AMERICAN CIRCUS      R  English   Action
   customer_id  staff_id  film_id  store_id         rental_date  \
0          431         2        1         1 2005-07-08 19:03:15   
1          518         1        1         1 2005-08-02 20:13:10   

          return_date  rental_duration  rental_id  
0 2005-07-11 21:29:15            114.0       4863  
1 2005-08-11 21:35:10            124.0      11433  
   staff_id  store_id    staff_name                         email
0         1         1  Mike Hillyer  Mike.Hillyer@sakilastaff.com
1         2         2  Jon Stephens  Jon.Stephens@sakilastaff.com
   customer_id   customer_name  active        customer_address customer_city  \
0          218      VERA MCCOY       1  1168 Najafabad Parkway         Kabul   
1          441  MARIO CHEATHAM       1  1924 Shimonoseki Drive         Batna   

  customer_country  
0      Afghanistan  
1 

# 2. Machine Learning

In [12]:
# Define training and test variable
X = fact_rent[['film_id','store_id']]
y = fact_rent['rental_duration']

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)

## 2.1 k-Nearest Neighbors Algorithm

In [13]:
# Create a k-NN classifier
knn = KNeighborsClassifier()

# Fit the classifier to the data
knn.fit(X_train, y_train)

# Compute and print score
print('knn training score: {}'.format(knn.score(X_train, y_train)))
print('knn test score: {}'.format(knn.score(X_test, y_test)))

knn training score: 0.847255369928401
knn test score: 0.7166666666666667


## 2.2 Gradient Boosting regression

In [14]:
# Instantiate Gradient Boosting regression
gbr = GradientBoostingRegressor(random_state=0)

# Fit the classifier to the data
gbr.fit(X_train, y_train)

# Compute and print score
print('gbr training score: {}'.format(gbr.score(X_train, y_train)))
print('gbr test score: {}'.format(gbr.score(X_test, y_test)))

gbr training score: 0.708298502705627
gbr test score: 0.5792198604170334
